# <a name="top_app"></a> Bloomberg Data Analytics: <br><span style="color:orange"> Using the Bloomberg Query Language with Equity Data: Basics </span>

<a id='s0'></a>
_Note: The examples in this notebook use the PyBQL API object model to create and execute BQL requests. For more information on getting started with BQL and the BQL object model, see <a href = "#Related-Resources"> Related Resources</a>._
### Topics
1. [__Key Benefits__](#s1)<a href='#s1'></a>
2. [__Basic Features__](#s2)<a href='#s2'></a>
- [2.1 How to Start](#s2.1)<a href='#s2.1'></a>
- [2.2 Fundamentals Rationalization](#s2.2)<a href='#s2.2'></a>
    - [2.2.1 Historical/Estimate alignment](#s2.2.1)<a href='#s2.2.1'></a>
    - [2.2.2 Estimate Time Series](#s2.2.2)<a href='#s2.2.2'></a>
    - [2.2.3 Calendarisation & Blending](#s2.2.3)<a href='#s2.2.3'></a>
- [2.3 Point in Time Data](#s2.3)<a href='#s2.3'></a>
- [2.4 Custom calculations](#s2.4)<a href='#s2.4'></a>
- [2.5 Dividends](#s2.5)<a href='#s2.5'></a>
3. [__Earnings Analysis__](#s3)<a href='#s3'></a>
- [3.1 Surprise Analysis](#s3.1)<a href='#s3.1'></a>
- [3.2 Analyst Sentiment](#s3.2)<a href='#s3.2'></a>
- [3.3 Custom Broker Estimates](#s3.3)<a href='#s3.3'></a>
4. [__Universes__](#s4)<a href='#s4'></a>
- [4.1 Multiple Securities in List](#s4.1)<a href='#s4.1'></a>
- [4.2 Index Members](#s4.2)<a href='#s4.2'></a>
- [4.3 Portfolio Members](#s4.3)<a href='#s4.3'></a>
- [4.4 Filtering](#s4.4)<a href='#s4.4'></a>
- [4.5 Linking to other assets](#s4.5)<a href='#s4.5'></a>
- [4.6 EquitiesUniv](#s4.6)<a href='#s4.6'></a>

<a id='s1'></a>
<span style="color:darkorange; font-size:2em"> 1 Key Benefits </span>

<div style="background-color: #232323; padding: 10px 25px">
- Faster, more efficient execution in Bloomberg Cloud<br>
- Clean, standardized data for cross asset analysis<br>
- Screening of an equities universe for stocks with certain characteristics<br>
- Cross-asset signals for equities that reference bond yields, vol skew, and options volume<br>
- Data sets spanning financials, estimates, consensus, revisions, technical analysis, and environmental, social, and governmental (ESG) data<br>
</div>

<img src="../../Visualisations/BQL Process.jpg" style="width: 500px;"/>

<a id='s2'></a>
<span style="color:darkorange; font-size:2em"> 2 Basic features </span>

<a id='s2.1'></a>
### 2.1 How to Start 


Import package for BQL

In [30]:
import bql

Then connect to the BQL Service to access Bloomberg Databases by bql.Service()

In [31]:
# always have this in one cell and keep it on the top, because "bq" can be reused after
bq = bql.Service()

<a id='s2.2'></a>

### 2.2 Fundamentals Rationalization


- Alignment of reported and estimated data in BQL<br>
- GIves you the ability to use a single field for both reported and estimated data (e.g. you no longer need to use both `IS_EPS` and `BEST_EPS`)
- Point-in-time company financials data that can take into account release and revision dates<br>
- Data retrieval and analytics on data using both fiscal and calendar years<br><br>

<div style="background-color: #232323; padding: 10px 25px">
- There are number of parameters for fundamental data fields. Here are just some of the most commonly used ones. For a full detailed reference of parameters, please visit our <a href="https://blinks.bloomberg.com/screens/docs%202085646">Fundamentals Reference Guide</a>.<br><br> 

    
| Name| Parameter ID | Default Value | Enum Values | Example |
|---|---|---|---|---|
| Period Type | fpt | LTM | LTM/P/A/S/Q/BA/BS/BQ/BT  | fpt='A'; fpt='BA' |  
| Period Offset | fpo | - | integer , integer＋R＋period , integer＋F＋period | fpo='1'; fpo='-1' |
| Period Reference | fpr | - | Date (YYYY-MM-DD), Year (YYYY), Year＋Quarter (YYYYQX) | fpr='2019'; fpr='2019Q4'|
| Period Type Source | fpts | P | P/A/Q/S | fpt='BT'・fpts='A': This will calculate 12m blended forward based on next two annuals |  
| Adjustment for abnormal items | adj | N | Y/N | |
| Actuals/Estimates | ae | ae | AE/A/E | fpr='2018'・ae='E': This will return estimates for 2018 |
| Estimates Source | est_source | BST | BST/BLI/BPE/CGD | est_source='CGD' |
| Currency | currency | Reported Currency | - | Currency='JPY' |
| Corporate Actions | ca_adj | splits | full, raw, splits | ca_adj='full' |  
</div>

</a>[Return to Index ↑](#s0)<a href='#s0'></a>

<a id='s2.2.1'></a>
#### 2.2.1 Reported & Estimated alignment<br>

<img src="../../Visualisations/align.png" style="width: 500px;"/>

##### Q: How can I get a time series of annual fundamentals/estimates?

In [32]:
# Select company
equity = 'IBM US Equity'

# we can use fixed period referencing using 'fpr'
# we must use the bql range function to specify our start and end
# bql will automatically align our actual and estimates data
eps = bq.data.is_eps(fpr=bq.func.range(start='2015',end='2022'),fpt='A')

req = bql.Request(equity, {'EPS':eps})
res = bq.execute(req)
data = res[0].df()
data

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,CURRENCY,EPS
ID,,,,,
IBM US Equity,2018-02-27,2015-12-31,2020-10-27,USD,13.480000
IBM US Equity,2018-02-27,2016-12-31,2020-10-27,USD,12.430000
IBM US Equity,2020-02-25,2017-12-31,2020-10-27,USD,6.170000
IBM US Equity,2020-02-25,2018-12-31,2020-10-27,USD,9.569671
IBM US Equity,2020-02-25,2019-12-31,2020-10-27,USD,10.630000
IBM US Equity,2020-10-21,2020-12-31,2020-10-27,USD,7.588333
IBM US Equity,2020-10-21,2021-12-31,2020-10-27,USD,9.850000
IBM US Equity,2020-10-21,2022-12-31,2020-10-27,USD,10.763333


In [33]:
# Select company
equity = 'IBM US Equity'

# we can also use relative period referencing by using 'fpo'
eps = bq.data.is_eps(fpo=bq.func.range(start='-4Y',end='+3Y'),fpt='A')

req = bql.Request(equity, {'EPS':eps})
res = bq.execute(req)
data = res[0].df()
data

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,CURRENCY,EPS
ID,,,,,
IBM US Equity,2018-02-27,2015-12-31,2020-10-27,USD,13.480000
IBM US Equity,2018-02-27,2016-12-31,2020-10-27,USD,12.430000
IBM US Equity,2020-02-25,2017-12-31,2020-10-27,USD,6.170000
IBM US Equity,2020-02-25,2018-12-31,2020-10-27,USD,9.569671
IBM US Equity,2020-02-25,2019-12-31,2020-10-27,USD,10.630000
IBM US Equity,2020-10-21,2020-12-31,2020-10-27,USD,7.588333
IBM US Equity,2020-10-21,2021-12-31,2020-10-27,USD,9.850000
IBM US Equity,2020-10-21,2022-12-31,2020-10-27,USD,10.763333


In [34]:
# Select company
equity = 'IBM US Equity'

# we can perform not only data rtreival but transformations as well
# here we use BQL to calcualte the annual growth rates
eps = bq.data.is_eps(fpo=bq.func.range(start='-4Y',end='+3Y'),fpt='A').pct_diff()

req = bql.Request(equity, {'EPS':eps})
res = bq.execute(req)
data = res[0].df()
data

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,EPS
ID,,,,
IBM US Equity,2018-02-27,2015-12-31,2020-10-27,NaN
IBM US Equity,2018-02-27,2016-12-31,2020-10-27,-7.789318
IBM US Equity,2020-02-25,2017-12-31,2020-10-27,-50.362027
IBM US Equity,2020-02-25,2018-12-31,2020-10-27,55.100016
IBM US Equity,2020-02-25,2019-12-31,2020-10-27,11.080099
IBM US Equity,2020-10-21,2020-12-31,2020-10-27,-28.613986
IBM US Equity,2020-10-21,2021-12-31,2020-10-27,29.804524
IBM US Equity,2020-10-21,2022-12-31,2020-10-27,9.272420


</a>[Return to Index ↑](#s0)<a href='#s0'></a>

<a id='s2.2.2'></a>
#### 2.2.2 Estimate Time Series<br>

##### Q: How can I get a time series of quarterly estimates

In [35]:
# Select company
equity = 'IBM US Equity'

# we can switch 'fpt' to q and use the 'ae' override to tell BQL to not align reported and reported data
# it will ONLY return estimate values - this could be useful if we cant to calculate historical surprise
eps = bq.data.is_eps(fpo=bq.func.range(start='-4Q',end='+3Q'),fpt='Q',ae='E')

req = bql.Request(equity, {'EPS':eps})
res = bq.execute(req)
data = res[0].df()
data

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,CURRENCY,EPS
ID,,,,,
IBM US Equity,2019-10-16,2019-09-30,2020-10-27,USD,2.538182
IBM US Equity,2020-01-20,2019-12-31,2020-10-27,USD,4.257000
IBM US Equity,2020-04-18,2020-03-31,2020-10-27,USD,1.666000
IBM US Equity,2020-07-18,2020-06-30,2020-10-27,USD,1.846667
IBM US Equity,2020-10-13,2020-09-30,2020-10-27,USD,2.017500
IBM US Equity,2020-10-21,2020-12-31,2020-10-27,USD,2.127778
IBM US Equity,2020-10-20,2021-03-31,2020-10-27,USD,1.483333
IBM US Equity,2020-10-20,2021-06-30,2020-10-27,USD,2.155000


##### Q: How can I get a time series of the consensus revisions for an estimate

In [36]:
# Select company
equity = 'IBM US Equity'

# use 'fpr' to specify the year you are interested in makes sure to set the 'ae' paramter to e
# the dates parameter lets you engage the BQL Point in time database to create a time series of revisions
# between the dates specified. Since we dont get a revisioj every day of the year we use fill=prev to fill the gaps in the time series
eps = bq.data.is_eps(fpr='2020',fpt='A',ae='E',dates=bq.func.range(start='2019-01-01',end='2020-09-30'),fill='prev')

req = bql.Request(equity, {'EPS':eps})
res = bq.execute(req)
data = res[0].df()
#we are only showing the first 10 values in the time series
data.head(10)

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,CURRENCY,EPS
ID,,,,,
IBM US Equity,2018-12-17,2020-12-31,2019-01-01,USD,13.056667
IBM US Equity,2018-12-17,2020-12-31,2019-01-02,USD,13.056667
IBM US Equity,2019-01-03,2020-12-31,2019-01-03,USD,13.254000
IBM US Equity,2019-01-03,2020-12-31,2019-01-04,USD,13.254000
IBM US Equity,2019-01-05,2020-12-31,2019-01-05,USD,13.254000
IBM US Equity,2019-01-05,2020-12-31,2019-01-06,USD,13.254000
IBM US Equity,2019-01-05,2020-12-31,2019-01-07,USD,13.254000
IBM US Equity,2019-01-05,2020-12-31,2019-01-08,USD,13.254000
IBM US Equity,2019-01-05,2020-12-31,2019-01-09,USD,13.254000


</a>[Return to Index ↑](#s0)<a href='#s0'></a>

<a id='s2.2.3'></a>
#### 2.2.3 Calendarisation & Blending

<span style="color:darkorange; font-size:15px"> Calendarisation </span>

A new parameter for the fiscal year end (FYE) allows the alignment of financial data of companies following different fiscal year ends for
their financial reporting.<br>
For example, unlike all the other US retail giants Costo has a fiscal year end in August (the rest follow a Jan year end reporting period).<br>
If you look at fiscal periods you are not comparing apples to apples
BQL lets you query calendar quarterd which means the query below will the fiscal quarters that overlap the most with the
calendar quarter  request in this case we ask for 2016 Q4.


<img src="../../Visualisations/calendarize.jpg" style="width: 500px;"/>

In [37]:
# Select company
equity = 'COST US Equity'

# use '
sales = bq.data.sales_rev_turn(fpr='2016Q4',fpt='Q',fye='C1231')/10e6

req = bql.Request(equity, {'Sales':sales})
res = bq.execute(req)
data = res[0].df()
data

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,CURRENCY,Sales
ID,,,,,
COST US Equity,2017-12-21,2016-12-31,2020-10-27,USD,2809.9


<span style="color:darkorange; font-size:15px"> Blending <span style="color:white; font-size:15px">(Blended Annual)</span></span>

While the calendarization of financial reporting calendar across companies usually works well for aligning fiscal periods of<br>
companies that report on a `quarterly` basis, for companies reporting on a less frequent basis the misalignment of fiscal periods<br>
can still be significant. 
To tackl this BQL allows you to `blend` data from different fiscal periods reported by the company to create a new blended fiscal<br>
period which is a calculated by Bloomberg. For example, Vodafone's financial disclosure follows a fiscal year ending on March 31.<br>


<img src="../../Visualisations/blended.jpg" style="width: 500px;"/>

In [38]:
# Select company
equity = 'VOD LN Equity'

#This query will return the sales data for a Blended Annual period ending on December 31, 2015, calculated as the
#time-weighted average of the Vodafone's reported EBIT data from fiscal periods 2015A and 2016A, as shown above.
sales = bq.data.sales_rev_turn(fpr='2015',fpt='BA',fye='C1231')/10e6

req = bql.Request(equity, {'Sales':sales})
res = bq.execute(req)
data = res[0].df()
data

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,CURRENCY,Sales
ID,,,,,
VOD LN Equity,2018-05-15,2015-12-31,2020-10-27,EUR,5598.872711


<span style="color:darkorange; font-size:15px"> Blending <span style="color:white; font-size:15px">(Blended Trailing)</span></span>

BQL Fundamentals also offers a Blended Trailing period type which dynamically links the period end date of the blended data to
an observation date (or as-of-date).<br>
This is a useful feature that helps us create blended estimates

Here we provide an example with Vodafone as of the date 22-03-2017 to illustrate teh calculation 

<img src="../../Visualisations/blendedtrailing.jpg" style="width: 400px; height: 500px"/>

In [39]:
# Select company
equity = 'VOD LN Equity'

#This query will return the ebit for a Blended Trailing period Blended Trailing period ending one year
#forward (on 2018-03-22) from  (2017-03-22) this is so because we have set the 'fpo' parameter to 1.
#In this example, it will be calculated as a time-weighted average of 3% of EBIT estimate
# from 2017A and 97% of EBIT estimate from 2018A for Vodafone.
ebit = bq.data.ebit(fpo='1',fpt='BT',dates='2017-03-22')/10e6

req = bql.Request(equity, {'EBIT':ebit})
res = bq.execute(req)
data = res[0].df()
data

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,CURRENCY,EBIT
ID,,,,,
VOD LN Equity,2017-03-21,2018-03-22,2017-03-22,EUR,492.463801


In [40]:
# Select company
equity = 'VOD LN Equity'

# This example shows how to use BQL to get the blended forward 12M estimate
eps = bq.data.is_eps(fpo='1',fpt='BT',dates=bq.func.range('-1Y','0D'))

req = bql.Request(equity, {'EPS':eps})
res = bq.execute(req)
data = res[0].df()
data.tail(5) # use tail to show the last 5 values

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,CURRENCY,EPS
ID,,,,,
VOD LN Equity,2020-10-15,2021-10-23,2020-10-23,EUR,0.078546
VOD LN Equity,2020-10-15,2021-10-24,2020-10-24,EUR,0.078623
VOD LN Equity,2020-10-15,2021-10-25,2020-10-25,EUR,0.078699
VOD LN Equity,2020-10-15,2021-10-26,2020-10-26,EUR,0.078776
VOD LN Equity,2020-10-15,2021-10-27,2020-10-27,EUR,0.078852


</a>[Return to Index ↑](#s0)<a href='#s0'></a><br><br>

<a id='s2.3'></a>
### 2.3 Point in Time

A key feature of Equity Fundamental data in BQL is that it is fully point in time - reflecting the data as it was reported historically.<br>
This means you can go back in history and see data absent revisions. We have done this to allow BQL to be a robust tool for `Factor Analysis` and `Backtesting`
It also means that we properly align market data (like price or market capitalization) with fundamental data

We can access the point in time fundamental data base by using the `dates` parameter with a fundamental field

##### Q: How do we get a times series of the PE_RATIO for a company

In [41]:
# Select company
equity = 'AMZN US Equity'

pe_ratio = bq.data.pe_ratio(dates=bq.func.range('2019-09-01','2019-09-30'),fill='prev')

req = bql.Request(equity, {'PE':pe_ratio})
res = bq.execute(req)
data = res[0].df()
data.tail(5) # use tail to show the last 5 values

,AS_OF_DATE,REVISION_DATE,PERIOD_END_DATE,PE
ID,,,,
AMZN US Equity,2019-09-26,2019-07-26,2019-06-30,72.192531
AMZN US Equity,2019-09-27,2019-07-26,2019-06-30,71.595436
AMZN US Equity,2019-09-28,2019-07-26,2019-06-30,71.595436
AMZN US Equity,2019-09-29,2019-07-26,2019-06-30,71.595436
AMZN US Equity,2019-09-30,2019-07-26,2019-06-30,72.029461


##### <span style="color:darkorange">Q </span>: What was the trailing 12M sales of RIO Tinto on the 05-05-2017

In [42]:
# Select company
equity = 'RIO LN Equity'

sales = bq.data.sales_rev_turn(fpt='LTM',dates='2017-05-05',fill='prev')/10e6

req = bql.Request(equity, {'Sales':sales})
res = bq.execute(req)
data = res[0].df()
data

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,CURRENCY,Sales
ID,,,,,
RIO LN Equity,2017-02-08,2016-12-31,2017-05-05,USD,3378.1


##### <span style="color:darkorange">Q </span>: On what date have the Annual Earning results been publish for IBM over the last 10 years

In [43]:
# Select company
equity = 'IBM US Equity'

eps = bq.data.is_eps(fpt='A',fpo=bq.func.range('-10Y','0Y'),fs='LR')

req = bql.Request(equity, {'EPS':eps})
res = bq.execute(req)
data = res[0].df()
# use Pyton to extract the revision date and period end date columns
data[['REVISION_DATE','PERIOD_END_DATE']]

,REVISION_DATE,PERIOD_END_DATE
ID,,
IBM US Equity,2010-01-19,2009-12-31
IBM US Equity,2011-01-18,2010-12-31
IBM US Equity,2012-01-19,2011-12-31
IBM US Equity,2013-01-22,2012-12-31
IBM US Equity,2014-01-21,2013-12-31
IBM US Equity,2015-01-20,2014-12-31
IBM US Equity,2016-01-19,2015-12-31
IBM US Equity,2017-01-19,2016-12-31
IBM US Equity,2018-01-18,2017-12-31


</a>[Return to Index ↑](#s0)<a href='#s0'></a><br><br>


<a id='s2.4'></a>
### 2.4 Custom Calculations

One of the key benefits of BQL is that it gives you more flexibility to define custom factors.
- Create custom combinations of fields defined by you
- Perform calculations and historical analysis with your custom fields

<span style="color:darkorange; font-size:15px"> See the BQL Editor for field explanations, parameters and defaults </span>

##### <span style="color:darkorange">Example </span> : Stock upside potential for stocks

In [44]:
equity = 'IBM US Equity'

price = bq.data.px_last()
analyst_target = bq.data.best_target_price()

upside_potential = (analyst_target/price - 1)*100

req = bql.Request(equity, {'Return Potential':upside_potential})
res = bq.execute(req)
data = res[0].df()
data

,DATE,CURRENCY,Return Potential
ID,,,
IBM US Equity,2020-10-27,None,21.07913


##### <span style="color:darkorange">Time Series </span> : Create a custom time series of this calculation and show the percentage change of the upside potential over the past week

In [45]:
equity = 'IBM US Equity'

price = bq.data.px_last(dates=bq.func.range('-1W','0D'),fill='prev')
analyst_target = bq.data.best_target_price(dates=bq.func.range('-1W','0D'),fill='prev')

upside_potential = (analyst_target/price - 1)*100

req = bql.Request(equity, {'Return Potential':upside_potential})
res = bq.execute(req)
data = res[0].df()
data

,DATE,CURRENCY,Return Potential
ID,,,
IBM US Equity,2020-10-20,None,16.975473
IBM US Equity,2020-10-21,None,19.323929
IBM US Equity,2020-10-22,None,18.348307
IBM US Equity,2020-10-23,None,18.103448
IBM US Equity,2020-10-24,None,18.103448
IBM US Equity,2020-10-25,None,18.103448
IBM US Equity,2020-10-26,None,21.079130
IBM US Equity,2020-10-27,None,21.079130


##### <span style="color:darkorange">Q </span>: How do we define our own Enterprise Value

In [46]:
equity = 'AMZN US Equity'

mkt_cap = bq.data.cur_mkt_cap()
net_debt = bq.data.net_debt().znav()
minorities = bq.data.minority_noncontrolling_interest().znav()
pension_reserve = bq.data.bs_pension_rsrv().znav()
pref_equity = bq.data.bs_pfd_eqy().znav()
associates = bq.data.bs_invest_in_assoc_co().znav()

ev = (mkt_cap + net_debt + minorities  + pref_equity - associates)/10e6

req = bql.Request(equity, {'EV':ev},with_params={'currency':'USD'})
res = bq.execute(req)
data = res[0].df()
data

,DATE,CURRENCY,EV
ID,,,
AMZN US Equity,2020-10-27,USD,162565.314634


</a>[Return to Index ↑](#s0)<a href='#s0'></a>

<a id='s2.5'></a>

### 2.5 Dividends
<div style="background-color: #232323; padding: 10px 25px">


BQL introduces a new way to interact with Bloombergs historical dividends database

<img src="../../Visualisations/DVD.jpg" style="width: 500px; height:300px"/><br><br> 
There are number of parameters for fundamental data fields. Here are just some of the most commonly used ones. For a full detailed reference of parameters, please visit our <a href="https://blinks.bloomberg.com/screens/docs%202085646">Fundamentals Reference Guide</a>.<br><br> 

    
| Name| Description | Possible Values |
|---|---|---|
| DATES |  Takes a list or range of dates, either gregorian or relative.| RANGE(-1Y, 0D); [2016-07-27] | 
| CA_DATA_TYPE | Allows you to switch between searching on announced, effective, pay or record date. This controls how dividends are aggregated in the results. | ANNOUNCED_DATE; PAY_DATE; RECORD_DATE; EFFECTIVE_DATE |
| CURRENCY | Allows you to control the currency of the returned dividend. This will convert using the BGN rate on the effective date of the dividend. | USD;GBP | 
| DIVIDEND_TYPE | Allows you to control which dividends are included in the results. Options are regular, special or all. | REGULAR; SPECIAL; ALL | 
| CA_ADJ | Allows you to control whether results are returned adjusted for capital changes. Options are full, splits only or raw. | RAW; SPLITS; FULL | 

</div>

##### <span style="color:darkorange">Q </span>: How can I get Vodafone's dividends currency converted into USD for the past 3 years?

In [47]:
equity = 'VOD LN Equity'

dividends = bq.data.cash_divs(dates=bq.func.range('-3Y','0D'),currency='USD').dropna()

req = bql.Request(equity, {'Dividends':dividends})
res = bq.execute(req)
data = res[0].df()
data

,DATE,CURRENCY,Dividends
ID,,,
VOD LN Equity,2017-11-23,USD,0.057344
VOD LN Equity,2018-06-07,USD,0.120683
VOD LN Equity,2018-11-22,USD,0.055176
VOD LN Equity,2019-06-06,USD,0.046904
VOD LN Equity,2019-11-28,USD,0.049527
VOD LN Equity,2020-06-11,USD,0.050841


##### <span style="color:darkorange">Q </span>: How can I get the `total` regular dividends paid by IBM in the past 5 years?

In [48]:
equity = 'IBM US Equity'

dividends = bq.data.cash_divs(dates=bq.func.range('-5Y','0D'),dividend_type='regular').dropna().sum()

req = bql.Request(equity, {'Dividends':dividends})
res = bq.execute(req)
data = res[0].df()
data

,DATE,CURRENCY,Dividends
ID,,,
IBM US Equity,2020-08-07,USD,30.22


</a>[Return to Index ↑](#s0)<a href='#s0'></a><br><br>

<a id='s3'></a>

<span style="color:darkorange; font-size:2em"> 3 Earning Analysis </span>


<a id='s3.1'></a>

### 3.1 Surprise Analysis

##### <span style="color:darkorange">Q </span>: How can we compute the earning surprise for a company

In [49]:
equity = 'IBM US Equity'

reported = bq.data.is_comp_eps_adjusted(fpt='Q',fpr='2020Q2',ae='A')
estimated = bq.data.is_eps(fpt='Q',fpr='2020Q2',ae='E',adj='Y')

surprise = (reported/estimated - 1) * 100

req = bql.Request(equity, {'Surprise':surprise})
res = bq.execute(req)
data = res[0].df()
data

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,CURRENCY,Surprise
ID,,,,,
IBM US Equity,2020-07-28,2020-06-30,2020-10-27,None,3.054545



##### Next step is to do this over time

In [50]:
equity = 'IBM US Equity'

reported = bq.data.is_comp_eps_adjusted(fpt='Q',fpr=bq.func.range('2019Q1','2020Q2'),ae='A')
estimated = bq.data.is_eps(fpt='Q',fpr=bq.func.range('2019Q1','2020Q2'),ae='E',adj='Y')

surprise = (reported/estimated - 1) * 100

req = bql.Request(equity, {'Surprise':surprise})
res = bq.execute(req)
data = res[0].df()
data

,REVISION_DATE,PERIOD_END_DATE,AS_OF_DATE,CURRENCY,Surprise
ID,,,,,
IBM US Equity,2020-04-28,2019-03-31,2020-10-27,None,1.351351
IBM US Equity,2020-07-28,2019-06-30,2020-10-27,None,3.294714
IBM US Equity,2019-10-30,2019-09-30,2020-10-27,None,0.507390
IBM US Equity,2020-02-25,2019-12-31,2020-10-27,None,0.502071
IBM US Equity,2020-04-28,2020-03-31,2020-10-27,None,1.610676
IBM US Equity,2020-07-28,2020-06-30,2020-10-27,None,3.054545


</a>[Return to Index ↑](#s0)<a href='#s0'></a>



<a id='s3.2'></a>

### 3.2 Analyst Sentiment


In [51]:
equity = 'IBM US Equity'

eps = bq.data.is_comp_eps_adjusted(fpt='A',fpo='1')
contributor_count = bq.data.contributor_count(eps)
num_revisions_up = bq.data.contributor_revisions(eps,revision_type='NETUP',revision_window='20w')
pct_chg_in_esimate = bq.data.is_comp_eps_adjusted(fpt='A',fpo='1',dates=bq.func.range('-1Y','0D'),fill='prev').pct_chg()
fields = {'contributor_count':contributor_count,
          'num_revisions_up':num_revisions_up,
          'pct_chg_in_esimate':pct_chg_in_esimate}

req = bql.Request(equity, fields)
res = bq.execute(req)
data = bql.combined_df(res)
data[fields.keys()]

,contributor_count,num_revisions_up,pct_chg_in_esimate
ID,,,
IBM US Equity,19.0,2.0,-27.973849


</a>[Return to Index ↑](#s0)<a href='#s0'></a><br><br>


<a id='s3.3'></a>
### 3.3. Custom Broker Estimates

- Curate a custom consensus
- Understanding broker coverage and entitlements for a list of securities
- Look at revisions for a list of securities
- Analyse broker closeness to actuals upon release


<div style="background-color: #232323; padding: 10px 25px">
Previously, we were only able to define 3 estimate source for consensus data in BQL:<br><br>
- <span style="color:darkorange">BST </span> (Bloomberg BEST : Simple average of all validated broker estimates available on Terminal, including those that you are not entitled to This is the default value for EST_SOURCE)<br>
- <span style="color:darkorange">BLI </span> (Bloomberg Leading Indicator: Same as BST but only including those that updated their estimates in the last 28 days)<br>
- <span style="color:darkorange">BPE </span> (Bloomberg Post Event: Only including those that updated their estimates since last event (eg earnings release))<br><br>


<span style="color:darkorange">Now, you are able to customise the consensus data based on your/your firm's choice of brokers for more relevant analysis </span> <br>


Please be aware that everyone has different broker entitlements and you will only be able to retrieve broker specific estimates from those that you are entitled to.<br>
To check your entitlements and/or request entitlements, you can check the <span style="color:#4da6ff">EEB&#60;GO&#62; </span> page for Estimate Consensus Detail<br>
If you would like to request entitlements for specific brokers, you must run <span style="color:#4da6ff">ENTC&#60;GO&#62; </span>  or click link: <a href="https://blinks.bloomberg.com/screens/ENTC">Click here to Request Entitlements</a>
    
</div><br>

<img src="../../Visualisations/broker_metadata.jpg" style="width:500px"/>

##### <span style="color:darkorange">Q </span>:  Which brokers am I entitled to for BCVN SW Equity's FY1 EPS estimates?

In [52]:
equity = 'BKW SW Equity'

estimates = bq.data.is_comp_eps_adjusted(fpt='A',fpo='1',est_source='BROKERS_ALL_ACTIVE').dropna()

req = bql.Request(equity,{'Estimate':estimates},preferences={'addcols':'all'})

res = bq.execute(req)
data = res[0].df()
data[['FIRM_NAME','EST_SOURCE_SPECIFIED','Estimate']]

,FIRM_NAME,EST_SOURCE_SPECIFIED,Estimate
ID,,,
BKW SW Equity,Baader Helvea,BAA,4.49
BKW SW Equity,Mirabaud Securities,MBD,4.63


##### <span style="color:darkorange">Q </span>:  How can I see revenue estimates for FP FP Equity from DZB, NSR and JPM?

In [53]:
equity = 'FP FP Equity'

estimates = bq.data.sales_rev_turn(fpt='A',fpo='1',est_source=['DZB','NSR','JPM'])

req = bql.Request(equity,{'Estimate':estimates},preferences={'addcols':'all'})

res = bq.execute(req)
data = res[0].df()
data[['FIRM_NAME','EST_SOURCE_SPECIFIED','Estimate']]

,FIRM_NAME,EST_SOURCE_SPECIFIED,Estimate
ID,,,
FP FP Equity,JP Morgan,JPM,NaN
FP FP Equity,DZ Bank AG Research,DZB,1.570000e+11
FP FP Equity,New Street Research LLP,NSR,NaN


##### <span style="color:darkorange">Q </span>:  Which broker's estimate was the closest to the actual reported value?

In [54]:
equity = 'AAPL US Equity'

consensus = bq.data.sales_rev_turn(fpt='A',fpo='1',est_source='BST',AE='E')
estimate = bq.data.sales_rev_turn(fpt='A',fpo='1',est_source='BROKERS_ALL_ACTIVE',AE='E')

difference = ((estimate/consensus)-1) *100

req = bql.Request(equity,{'Deviation':difference},preferences={'addcols':'all'})

res = bq.execute(req)
data = res[0].df()
data[['EST_SOURCE_SPECIFIED','Deviation']].sort_values('Deviation',ascending=False).head()

,EST_SOURCE_SPECIFIED,Deviation
ID,,
AAPL US Equity,RHK,2.782136
AAPL US Equity,WMS,1.792309
AAPL US Equity,DZB,0.742360
AAPL US Equity,NSR,-0.013208
AAPL US Equity,CRO,-0.167181


</a>[Return to Index ↑](#s0)<a href='#s0'></a>

<a id='s4'></a>

<span style="color:darkorange; font-size:2em"> 4 Universes </span>
<br>

You don’t need to rewrite a request each time you want the same data for different IDs. You can simply create a universe that contains all of the IDs.


<a id='s4.1'></a>

### 4.1 Multiple Securities in List


In [55]:
# Define a list of securities for the query
securities = ['IBM US Equity', 'AAPL US Equity']

sector = bq.data.gics_sector_name()

req = bql.Request(securities, {'Sector':sector})
res = bq.execute(req)
data = res[0].df()
data

,Sector
ID,
IBM US Equity,Information Technology
AAPL US Equity,Information Technology


</a>[Return to Index ↑](#s0)<a href='#s0'></a>

<a id='s4.2'></a>

### 4.2 Index Members

<div style="background-color: #232323; padding: 10px 25px">
<span style="color:darkorange">- BQL enables us to retrieve the members of an index with bq.univ.members()</span><br>
- Membership access is in line with your terminal permissioning
<div>

In [56]:
index = bq.univ.members('INDU Index')

price = bq.data.px_last(fill='prev')
analyst_target = bq.data.best_target_price(fill='prev')

upside_potential = (analyst_target/price - 1)*100

req = bql.Request(index, {'Return Potential':upside_potential})
res = bq.execute(req)
data = res[0].df()
data.head()

,DATE,CURRENCY,Return Potential
ID,,,
AXP UN Equity,2020-10-27,None,12.620912
VZ UN Equity,2020-10-27,None,7.398298
BA UN Equity,2020-10-27,None,16.415329
CAT UN Equity,2020-10-27,None,-0.015981
JPM UN Equity,2020-10-27,None,14.313282


In [57]:
#Retrieving membership data as of a particular point
#Universe with dates parameter
universe = bq.univ.members('INDU Index', dates='2018-01-01')
#Get the Name of the securities
name = bq.data.name() 

req = bql.Request(universe, name)
res = bq.execute(req)
data = res[0].df().head(10)
data

,ID_DATE,NAME()
ID,,
AAPL UW Equity,2018-01-01,Apple Inc
AXP UN Equity,2018-01-01,American Express Co
BA UN Equity,2018-01-01,Boeing Co/The
CAT UN Equity,2018-01-01,Caterpillar Inc
CSCO UW Equity,2018-01-01,Cisco Systems Inc
CVX UN Equity,2018-01-01,Chevron Corp
DD UN Equity,2018-01-01,DuPont de Nemours Inc
DIS UN Equity,2018-01-01,Walt Disney Co/The
GE UN Equity,2018-01-01,General Electric Co


</a>[Return to Index ↑](#s0)<a href='#s0'></a>
<a id='s4.3'></a>
### 4.3 Portfolio Members
<div style="background-color: #232323; padding: 10px 25px">
<span style="color:darkorange">- Use the members function to also access the members of your portfolio using the portfolio ID.</span><br>
- the syntax is members('portfolio ID',type=PORT)<br><br>

<img src="../../Visualisations/portfolio.jpg" style="width: 700px;"/>
<div>

In [61]:
#Replace the portfolio ID (U10101389-782) with your own
portfolio = bq.univ.members('U10101389-782',type='PORT')

sec_id = bq.data.id()

req = bql.Request(portfolio, sec_id)
res = bq.execute(req)
data = res[0].df()
data.head()

,Weights,Positions,ORIG_IDS,ID()
ID,,,,
A UN Equity,0.311560,831.357313,U10101389-782,A UN Equity
AAP UN Equity,0.112091,206.962588,U10101389-782,AAP UN Equity
AAPL UW Equity,0.465301,1129.146940,U10101389-782,AAPL UW Equity
ABBV UN Equity,0.261092,868.623189,U10101389-782,ABBV UN Equity
ABC UN Equity,0.081340,230.857625,U10101389-782,ABC UN Equity


In [63]:
#If we want to see the weights, this is one of the columns associated with ID

portfolio = bq.univ.members('U10101389-782', type='PORT')
#Of the output, select the columns called 'WEIGHTS'
weights = bq.data.ID()['WEIGHTS']

req = bql.Request(portfolio, weights)
res = bq.execute(req)

data = res[0].df()
data.head()

,ID().WEIGHTS
ID,
A UN Equity,0.311560
AAP UN Equity,0.112091
AAPL UW Equity,0.465301
ABBV UN Equity,0.261092
ABC UN Equity,0.081340


</a>[Return to Index ↑](#s0)<a href='#s0'></a>
<a id='s4.4'></a>
### 4.4 Filtering


- BQL allows us to screen our universe directly on the Bloomberg Server. We can access this with bq.univ.filter()<br><br>
<img src="../../Visualisations/BQL Filtering.jpg" style="width: 600px;"/>


##### <span style="color:darkorange">Q </span>:  How do you filter by a single condition?

In [73]:
#lets only retreive the financials companies in the DOW Jones index
univ = bq.univ.members('INDU Index')
sector = bq.data.gics_sector_name() 
screen = bq.univ.filter(univ, sector == 'Financials')

#Define our request
req = bql.Request(screen, sector)
res = bq.execute(req)

data = res[0].df()
data

,GICS_SECTOR_NAME()
ID,
AXP UN Equity,Financials
JPM UN Equity,Financials
TRV UN Equity,Financials
GS UN Equity,Financials


##### <span style="color:darkorange">Q </span>:  How do you filter by multiple conditions?

In [80]:
#lets retreive the financials companies in the DOW Jones index
#that had a total return in 2019 greater than 30%
univ = bq.univ.members('INDU Index')
sector = bq.data.gics_sector_name()
total_return = bq.data.total_return(calc_interval=bq.func.range('2018-12-31','2019-12-31'))
criteria = bq.func.and_(sector == 'Financials', total_return > 0.3)
screen = bq.univ.filter(univ, criteria)

#Define our request
req = bql.Request(screen, total_return)
res = bq.execute(req)

data = res[0].df()
data

,"TOTAL_RETURN(calc_interval=RANGE(2018-12-31,2019-12-31))"
ID,
AXP UN Equity,0.325240
JPM UN Equity,0.472668
GS UN Equity,0.404637


</a>[Return to Index ↑](#s0)<a href='#s0'></a>


<a id='s4.5'></a>
### 4.5 Linking to other assets

In [87]:
#Using an equity ticker we can query information about its bonds
#we do thi sby using the 'bonds' universe function
bonds = bq.univ.bonds(['VOD LN Equity'])
spread = bq.data.spread()

#execcute request
req = bql.Request(bonds, {'Spread':spread})
res = bq.execute(req)

#let's just see the first 10
data = bql.combined_df(res)
data.head(10)

,DATE,Spread
ID,,
ZO136566 Corp,2020-10-27,343.159729
ZO136474 Corp,2020-10-27,316.333313
AX916964 Corp,2020-10-27,362.133881
AU654357 Corp,2020-10-27,348.675903
AU654481 Corp,2020-10-27,297.883881
AS779475 Corp,2020-10-27,73.319016
AQ340827 Corp,2020-10-27,156.587509
AS779498 Corp,2020-10-27,111.813507
ZR549306 Corp,2020-10-27,218.141190


</a>[Return to Index ↑](#s0)<a href='#s0'></a>
<a id='s4.6'></a>
### 4.6 Equities Universe

- An extremely powerful feature of BQL is the ability to interogate the entire equities database on Bloomberg
- We can access this using `bq.univ.equitiesuniv()` and it should always be use in conjunction with `bq.univ.filter()`<br><br>

In [91]:
#lets find out how many companies are listed on the LSE
#and in which sectors
univ = bq.univ.equitiesuniv(['ACTIVE','PRIMARY'])
sector = bq.data.gics_sector_name()
screen = bq.univ.filter(univ, exchange == 'LN')

#define our output as the count of securities
#find out more about group in the next section
num_securities = bq.data.id().group(sector).count()

#Define our request
req = bql.Request(screen, num_securities,with_params={'mode':'cached'})
res = bq.execute(req)

data = res[0].df()
data

,ORIG_IDS,GICS_SECTOR_NAME(),"COUNT(GROUP(ID(),GICS_SECTOR_NAME()))"
ID,,,
Communication Services,None,Communication Services,78
Consumer Discretionary,None,Consumer Discretionary,156
Consumer Staples,None,Consumer Staples,55
Energy,None,Energy,113
Financials,None,Financials,207
Health Care,None,Health Care,105
Industrials,None,Industrials,215
Information Technology,None,Information Technology,154
Materials,None,Materials,152


</a>[Return to Index ↑](#s0)<a href='#s0'></a>
<a id='s5'></a>

<span style="color:green; font-size:2em"> 5. BQL Equity Basics Summary </span>



### BQL

- Custom calculations: combining fields, applying BQL functions, using new BQL metrics


### Benefits
- Save time and effort cleaning, manipulating, sorting data
- Data-efficient analysis
- Focus on generating ideas and alpha

<span style="color:yellow; font-size:2em"> Congratulations - What's Next? </span>

- This is the beginning: Use BQL to dig deep and conduct in-depth cross-asset financial data analysis
- Regular Updates for future releases: Webinars, BQNT Spotlight, BQNT Whatsnew
- Connection to other Bloomberg services, eg Portfolio integration (PORT), order management systems (AIM, TOMS), proprietary data (CDE)

----

<p style="text-align:center;">
    Click on the links below to continue learning.<br>
    <a href="../Introduction/BQuant Basics - Equity.ipynb">&larr; Back to the Introduction</a>&emsp;&emsp;
    <a href="#s0">&uarr; Back to Top </a>&emsp;&emsp;
    <a href="2.2 Equity Advanced.ipynb">Next Topic: Equity Advanced &rarr;</a>
    <br>

</p>
